# High-level LSTM CNTK Example

In [16]:
print("Not working for some reason ...")

Not working for some reason ...


In [2]:
import numpy as np
import os
import sys
import cntk
from cntk.layers import Embedding, LSTM, Dense, Recurrence
from cntk import sequence
from common.params_lstm import *
from common.utils import *

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.1
CNTK:  2.1
GPU:  ['Tesla K80', 'Tesla K80']


In [4]:
def create_symbol():
    # Weight initialiser from uniform distribution
    # Activation (unless states) is None
    with cntk.layers.default_options(init = cntk.glorot_uniform()):
        x = Embedding(NUMHIDDEN)(features)
        # Fold same as recurrence but returns only final-state
        x = Recurrence(LSTM(NUMHIDDEN))(x)
        x = sequence.last(x)
        x = Dense(1)(x)
        return x

In [5]:
def init_model(m):
    # Loss (dense labels); check if support for sparse labels
    loss = cntk.binary_cross_entropy(m, labels)  
    # ADAM
    #Alpha is the learning_rate
    #Beta1 is momentum parameter
    #Beta2 is variance_momentum parameter  
    learner = cntk.adam(m.parameters,
                        lr=cntk.learning_rate_schedule(LR, cntk.UnitType.minibatch) ,
                        momentum=cntk.momentum_schedule(BETA_1), 
                        variance_momentum=cntk.momentum_schedule(BETA_2),
                        epsilon=EPS,
                        unit_gain=False)
    trainer = cntk.Trainer(m, (loss, cntk.classification_error(m, labels)), [learner])
    return trainer

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES)# CNTK format
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Trimming to 25000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000,) (25000,)
int32 int32 float32 float32
CPU times: user 5.58 s, sys: 360 ms, total: 5.94 s
Wall time: 6.99 s


In [13]:
%%time
# Placeholders
features = sequence.input_variable(shape=MAXFEATURES, is_sparse=False)
labels = cntk.input_variable(1)
# Load symbol
sym = create_symbol()

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 11.7 ms


In [14]:
%%time
trainer = init_model(sym)

CPU times: user 36 ms, sys: 4 ms, total: 40 ms
Wall time: 41.1 ms


In [15]:
%%time
# Train model
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        trainer.train_minibatch({features: data, labels: label})
    # Log (this is just last batch in epoch, not average of batches)
    eval_error = trainer.previous_minibatch_evaluation_average
    print("Epoch %d  |  Accuracy: %.6f" % (j+1, (1-eval_error)))

/anaconda/envs/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "int32", but your input variable (uid "Input1633") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


ValueError: The trailing dimensions of the Value shape '[64 x 150]' do not match the Variable 'Input('Input1633', [#, *], [25000])' shape '[25000]'.

[CALL STACK]
[0x7f4610eb35ec]                                                       + 0x6735ec
[0x7f4610f8589a]    CNTK::Utils::  VerifyVariableValueCompatibility  (CNTK::Variable const&,  std::shared_ptr<CNTK::Value> const&,  CNTK::NDShape*) + 0xb0a
[0x7f4610efaf02]    CNTK::CompositeFunction::  InferFreeDimensionsOfArguments  (std::unordered_map<CNTK::Variable,std::shared_ptr<CNTK::Value>,std::hash<CNTK::Variable>,std::equal_to<CNTK::Variable>,std::allocator<std::pair<CNTK::Variable const,std::shared_ptr<CNTK::Value>>>> const&) + 0x142
[0x7f4610eff88e]    CNTK::CompositeFunction::  Forward  (std::unordered_map<CNTK::Variable,std::shared_ptr<CNTK::Value>,std::hash<CNTK::Variable>,std::equal_to<CNTK::Variable>,std::allocator<std::pair<CNTK::Variable const,std::shared_ptr<CNTK::Value>>>> const&,  std::unordered_map<CNTK::Variable,std::shar